In [1]:
# cd ..

In [2]:
import s3fs
import pyarrow.parquet as pq
# import missingno as msno


s3 = s3fs.S3FileSystem()

df = pq.ParquetDataset('s3://airbnb-barcelona/valid/currentDate=2020-03-11', filesystem=s3).read_pandas().to_pandas()

In [3]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
def removeRowsWithValues(df , col, values):
    return df[~df[col].isin(values)]

In [5]:
# df.head()

In [6]:
# has_availability: True, same value
# license: TODO
# amenities: TODO
# property_type: TODO
# host_verifications: TODO

In [7]:
def only_barcelona_market():
    return df.loc[df['market'] == "Barcelona"]
df = only_barcelona_market()

In [8]:
def drop_columns():
    dropped_columns = ["first_review", "last_review", "has_availability", "market",'rowId','id','host_location','host_neighbourhood','street','neighbourhood','neighbourhood_cleansed','calendar_updated','license', 'amenities','property_type','zipcode','neighbourhood_group_cleansed','host_verifications','host_since','reviews_per_month']
    return df.drop(dropped_columns, axis=1)

In [9]:
df.shape

(20406, 66)

In [10]:
dropped_cols_df = drop_columns()
dropped_cols_df.shape

(20406, 46)

In [11]:
def drop_rows():
    dropped_cols_rows_df = dropped_cols_df.dropna(subset=["bedrooms", "host_listings_count","host_total_listings_count"])
    
    dropped_host_response_time = ["ES","el Barri Gòtic"]

    dropped_cols_rows_df = removeRowsWithValues(dropped_cols_rows_df, "host_response_time", dropped_host_response_time)

    # dropped_cols_rows_df = removeRowsWithValues(dropped_cols_rows_df, "license", ["Example"])
    return dropped_cols_rows_df

dropped_cols_rows_df = drop_rows()

In [12]:
dropped_cols_rows_df.shape

(20365, 46)

In [13]:
def fill_missing_data():
    values = {'security_deposit': 0, 'cleaning_fee': 0, "host_response_time":"N/A", "host_response_rate": 0, "host_has_profile_pic": 0, "host_identity_verified": 0, "host_is_superhost": 0}
    return dropped_cols_rows_df.fillna(value=values)

dropped_cols_rows_df = fill_missing_data()

In [14]:
# dropped_cols_rows_df["host_total_listings_count"].isna().sum() 


In [15]:
# dropped_cols_rows_df["host_total_listings_count"].value_counts()

In [16]:
# df["neighbourhood_group_cleansed"].value_counts()

In [17]:
# dropped_cols_rows_df["security_deposit"].value_counts()

In [18]:
cols = ["host_has_profile_pic","host_identity_verified"]
for col in cols:
    dropped_cols_rows_df[col] = pd.to_numeric(dropped_cols_rows_df[col]).astype('int64')
# dropped_cols_rows_df["host_has_profile_pic"] = pd.to_numeric(dropped_cols_rows_df["host_has_profile_pic"])
# dropped_cols_rows_df["host_has_profile_pic"].value_counts()

dropped_cols_rows_df["host_identity_verified"].value_counts()

0    13871
1     6494
Name: host_identity_verified, dtype: int64

In [19]:
def convert_boolean_to_float():
    cols = ["host_has_profile_pic","host_identity_verified", "host_is_superhost", "is_location_exact", "instant_bookable", "require_guest_profile_picture", "require_guest_phone_verification"]
    for col in cols:
        dropped_cols_rows_df[col] = pd.to_numeric(dropped_cols_rows_df[col]).astype('int64')

convert_boolean_to_float()

In [20]:
def convert_col_with_value_to_one(df, col):
    df[col]

In [21]:
dropped_cols_rows_df.head()

,host_response_time,host_response_rate,host_is_superhost,host_listings_count,host_total_listings_count,host_has_profile_pic,host_identity_verified,latitude,longitude,is_location_exact,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms
0,within an hour,0.99,0,46.0,46.0,1,1,41.40889,2.18555,1,Entire home/apt,6.0,1.0,2.0,4.0,Real Bed,130.0,150.0,42.0,2.0,25.0,3.0,730.0,3.0,730.0,0.0,0.0,29.0,304.0,1.0,0.0,80.0,10.0,10.0,2.0,10.0,10.0,8.0,0,flexible,0,0,30.0,30.0,0.0,0.0
1,within an hour,0.99,0,46.0,46.0,1,1,41.40420,2.17306,1,Entire home/apt,8.0,2.0,3.0,6.0,Real Bed,60.0,150.0,50.0,2.0,30.0,1.0,1125.0,3.9,1125.0,15.0,24.0,50.0,312.0,20.0,15.0,87.0,9.0,9.0,10.0,10.0,9.0,9.0,1,strict_14_with_grace_period,0,0,30.0,30.0,0.0,0.0
2,within an hour,1.00,1,5.0,5.0,1,1,41.41203,2.22114,0,Entire home/apt,6.0,2.0,3.0,5.0,Real Bed,210.0,300.0,80.0,3.0,10.0,3.0,30.0,3.1,1125.0,29.0,59.0,89.0,359.0,51.0,17.0,95.0,10.0,10.0,10.0,10.0,9.0,9.0,1,strict_14_with_grace_period,0,1,2.0,2.0,0.0,0.0
3,within an hour,1.00,1,1.0,1.0,1,1,41.40145,2.15645,1,Private room,2.0,1.0,1.0,1.0,Real Bed,32.0,0.0,0.0,1.0,25.0,1.0,730.0,1.0,730.0,13.0,13.0,13.0,46.0,268.0,44.0,95.0,10.0,9.0,10.0,10.0,10.0,10.0,1,strict_14_with_grace_period,1,1,1.0,0.0,1.0,0.0
4,within an hour,0.92,0,39.0,39.0,1,0,41.40950,2.15938,1,Entire home/apt,4.0,1.0,1.0,1.0,Real Bed,60.0,200.0,58.0,4.0,0.0,1.0,27.0,2.1,27.0,16.0,33.0,58.0,324.0,182.0,32.0,92.0,9.0,9.0,8.0,9.0,9.0,9.0,1,strict_14_with_grace_period,0,0,39.0,39.0,0.0,0.0


In [22]:
dropped_cols_rows_df.host_identity_verified.value_counts()

0    13871
1     6494
Name: host_identity_verified, dtype: int64

In [23]:
import numpy as np
from sklearn.impute import SimpleImputer
def fill_missing_data_with_mean():
    cols_filled_with_nums = ["review_scores_rating","review_scores_accuracy","review_scores_cleanliness","review_scores_checkin","review_scores_communication","review_scores_location","review_scores_value"]
    imp = SimpleImputer(missing_values=np.nan, strategy='mean')
    dropped_cols_rows_df[cols_filled_with_nums] = imp.fit_transform(dropped_cols_rows_df[cols_filled_with_nums])
    return

fill_missing_data_with_mean()

In [24]:
def one_hot_encoding():
    dropped_cols_rows_encoding_df = pd.get_dummies(dropped_cols_rows_df, columns=['host_response_time'], prefix = ['host_response_time'])
    dropped_cols_rows_encoding_df = pd.get_dummies(dropped_cols_rows_encoding_df, columns=['bed_type'], prefix = ['bed_type'])
    dropped_cols_rows_encoding_df = pd.get_dummies(dropped_cols_rows_encoding_df, columns=['room_type'], prefix = ['room_type'])
    dropped_cols_rows_encoding_df = pd.get_dummies(dropped_cols_rows_encoding_df, columns=['cancellation_policy'], prefix = ['cancellation_policy'])
    return dropped_cols_rows_encoding_df
    

In [25]:
dropped_cols_rows_encoding_df = one_hot_encoding()


In [26]:
dropped_cols_rows_encoding_df.head()

,host_response_rate,host_is_superhost,host_listings_count,host_total_listings_count,host_has_profile_pic,host_identity_verified,latitude,longitude,is_location_exact,accommodates,bathrooms,bedrooms,beds,price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,host_response_time_N/A,host_response_time_a few days or more,host_response_time_within a day,host_response_time_within a few hours,host_response_time_within an hour,bed_type_Airbed,bed_type_Couch,bed_type_Futon,bed_type_Pull-out Sofa,bed_type_Real Bed,room_type_Entire home/apt,room_type_Hotel room,room_type_Private room,room_type_Shared room,cancellation_policy_flexible,cancellation_policy_moderate,cancellation_policy_strict,cancellation_policy_strict_14_with_grace_period,cancellation_policy_super_strict_30,cancellation_policy_super_strict_60
0,0.99,0,46.0,46.0,1,1,41.40889,2.18555,1,6.0,1.0,2.0,4.0,130.0,150.0,42.0,2.0,25.0,3.0,730.0,3.0,730.0,0.0,0.0,29.0,304.0,1.0,0.0,80.0,10.0,10.0,2.0,10.0,10.0,8.0,0,0,0,30.0,30.0,0.0,0.0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0
1,0.99,0,46.0,46.0,1,1,41.40420,2.17306,1,8.0,2.0,3.0,6.0,60.0,150.0,50.0,2.0,30.0,1.0,1125.0,3.9,1125.0,15.0,24.0,50.0,312.0,20.0,15.0,87.0,9.0,9.0,10.0,10.0,9.0,9.0,1,0,0,30.0,30.0,0.0,0.0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0
2,1.00,1,5.0,5.0,1,1,41.41203,2.22114,0,6.0,2.0,3.0,5.0,210.0,300.0,80.0,3.0,10.0,3.0,30.0,3.1,1125.0,29.0,59.0,89.0,359.0,51.0,17.0,95.0,10.0,10.0,10.0,10.0,9.0,9.0,1,0,1,2.0,2.0,0.0,0.0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0
3,1.00,1,1.0,1.0,1,1,41.40145,2.15645,1,2.0,1.0,1.0,1.0,32.0,0.0,0.0,1.0,25.0,1.0,730.0,1.0,730.0,13.0,13.0,13.0,46.0,268.0,44.0,95.0,10.0,9.0,10.0,10.0,10.0,10.0,1,1,1,1.0,0.0,1.0,0.0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0
4,0.92,0,39.0,39.0,1,0,41.40950,2.15938,1,4.0,1.0,1.0,1.0,60.0,200.0,58.0,4.0,0.0,1.0,27.0,2.1,27.0,16.0,33.0,58.0,324.0,182.0,32.0,92.0,9.0,9.0,8.0,9.0,9.0,9.0,1,0,0,39.0,39.0,0.0,0.0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
